### Quick Start

We have setup a kind cluster already. This will be our "host cluster" 

Now let’s list the nodes in host cluster.

In [3]:
! kubectl get nodes -o wide

NAME                              STATUS   ROLES                  AGE   VERSION   INTERNAL-IP   EXTERNAL-IP   OS-IMAGE       KERNEL-VERSION                      CONTAINER-RUNTIME
kcd-vcluster-demo-control-plane   Ready    control-plane,master   67m   v1.23.6   172.18.0.2    <none>        Ubuntu 21.10   5.10.16.3-microsoft-standard-WSL2   containerd://1.6.4
kcd-vcluster-demo-worker          Ready    <none>                 66m   v1.23.6   172.18.0.4    <none>        Ubuntu 21.10   5.10.16.3-microsoft-standard-WSL2   containerd://1.6.4
kcd-vcluster-demo-worker2         Ready    <none>                 66m   v1.23.6   172.18.0.5    <none>        Ubuntu 21.10   5.10.16.3-microsoft-standard-WSL2   containerd://1.6.4
kcd-vcluster-demo-worker3         Ready    <none>                 66m   v1.23.6   172.18.0.3    <none>        Ubuntu 21.10   5.10.16.3-microsoft-standard-WSL2   containerd://1.6.4


Let's list the namespaces in the host cluster.

In [4]:
! kubectl get namespace

NAME                 STATUS   AGE
default              Active   69m
kube-node-lease      Active   69m
kube-public          Active   69m
kube-system          Active   69m
local-path-storage   Active   69m
metallb-system       Active   68m


Let's create a namespace called "host-namespace" in the underlying host cluster which is supposed to host our virtual cluster.

In [5]:
! kubectl create namespace host-namespace

namespace/host-namespace created


Now let's create a vcluster on top of the host cluster inside the namespace host-namespace. 


Let's use vcluster cli to create vcluster with the name vc1 in the host namespace. 

In [6]:
! vcluster create vc1 -n host-namespace


[info]   execute command: helm upgrade vc1 vcluster --repo https://charts.loft.sh --version 0.8.1 --kubeconfig /tmp/4089102180 --namespace host-namespace --install --repository-config='' --values /tmp/352538868
[done] √ Successfully created virtual cluster vc1 in namespace host-namespace. 
- Use 'vcluster connect vc1 --namespace host-namespace' to access the virtual cluster
- Use `vcluster connect vc1 --namespace host-namespace -- kubectl get ns` to run a command directly within the vcluster


Then we run the vcluster list to see that vcluster is listed.

In [7]:
! vcluster list


 NAME   NAMESPACE        CREATED                         AGE  
 vc1    host-namespace   2022-05-27 16:24:53 +0530 IST   29s  



Let's see what's inside the host-namespace. 

In [8]:
! kubectl get all -n host-namespace

NAME                                               READY   STATUS    RESTARTS   AGE
pod/coredns-5df468b6b7-cndph-x-kube-system-x-vc1   1/1     Running   0          59s
pod/vc1-0                                          2/2     Running   0          73s

NAME                                         TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)                  AGE
service/kube-dns-x-kube-system-x-vc1         ClusterIP   10.96.246.96   <none>        53/UDP,53/TCP,9153/TCP   59s
service/vc1                                  ClusterIP   10.96.90.227   <none>        443/TCP                  73s
service/vc1-headless                         ClusterIP   None           <none>        443/TCP                  73s
service/vc1-node-kcd-vcluster-demo-worker2   ClusterIP   10.96.53.194   <none>        10250/TCP                59s

NAME                   READY   AGE
statefulset.apps/vc1   1/1     73s


Lets connect to the vcluster using the vcluster connect command. Under the hood it runs "kubectl port forward" to connect to the vcluster's API server

Run this command
```shell
    vcluster connect vc1 --namespace host-namespace
```
in your terminal. Make sure you set Demo1_Quick_Start as the current directory before running the comand

Observe that vcluster cli connect command also creates a kubeconfig file in the current working directory which can be used to access the vcluster

In all the subsequent commands, notice that we have either exported the KUBECONFIG environment variable or passed the kubeconfig file explicitly to kubectl so that all kubectl commands
being run now point to the vcluster's api server and not the host cluster's api server

In [15]:
%%bash 
export KUBECONFIG=./kubeconfig.yaml
echo "Namespaces:"
kubectl get namespace

echo -e "\n\nPods:"
kubectl get pods -n kube-system


Namespaces:
NAME              STATUS   AGE
default           Active   10m
kube-system       Active   10m
kube-public       Active   10m
kube-node-lease   Active   10m


Pods:
NAME                       READY   STATUS    RESTARTS   AGE
coredns-5df468b6b7-cndph   1/1     Running   0          10m


Lets now create a sample workload on the vcluster in namespace ns1

In [24]:
! kubectl create namespace ns1 --kubeconfig ./kubeconfig.yaml


namespace/ns1 created


In [25]:
! kubectl create deployment sample -n ns1 --image=nginx --replicas 2 --kubeconfig ./kubeconfig.yaml

sample   0/2     0            0           0s
deployment.apps/sample created
sample   0/2     0            0           0s
sample   0/2     0            0           1s
sample   0/2     2            0           1s
sample   1/2     2            1           4s
sample   2/2     2            2           6s


In [26]:
! kubectl get deploy -n ns1 --kubeconfig ./kubeconfig.yaml

NAME     READY   UP-TO-DATE   AVAILABLE   AGE
sample   2/2     2            2           12s


In [27]:
! kubectl get pods -n ns1 --kubeconfig ./kubeconfig.yaml

NAME                      READY   STATUS    RESTARTS   AGE
sample-7f75ccc884-9d5zv   1/1     Running   0          33s
sample-7f75ccc884-p28cs   1/1     Running   0          32s


Now our vcluster seems to be running like a regular kubernetes cluster.

#### Cleanup

In [ ]:
! vcluster delete vc1 -n host-namespace
! kubectl delete ns host-namespace

In [ ]:
! kubectl get ns